## 1. 라이브러리

In [1]:

import olefile
import glob
import pandas as pd
import os
from datetime import datetime as dt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from konlpy.tag import Okt

## 2. INPUT 파트

In [2]:
# !pip install olefile
# !pip install docx
# !pip install exceptions
# !pip install python-pptx
# !pip install python-docx
# !pip install tika

In [3]:
# connect google drive
# from google.colab import drive
# drive.mount('/content/drive')

#### 2-1. hwp, pptx, docx, xls, csv, xlsx. pdf -> txt로 변환

In [4]:
#리스트 -> 문자열
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

In [5]:
# 정규표현식 한글 남겨두고 특수문자, \r\n 제거하여 관련 정규문
import re

def remove_special_characters(text):
    return re.sub(r'[^0-9a-zA-Z가-힣\s\r\n]', '', text).strip()

In [6]:
pwd

'c:\\Users\\user\\Desktop\\Python\\Project\\Project3\\prototype\\data'

In [7]:
target_path = input("검사하실 target문서의 directory경로를 입력하시요: ")
compare_path = input("문서 뭉치 directory경로를 입력하시오: ")

In [8]:
#검사하실 target문서 directory 경로
# hwp 확장자 dataFrame으로 변환
filelist_hwp = glob.glob(f'{target_path}/'+'*.hwp')


hwp_df = pd.DataFrame()
hwp_df['title'] = np.nan
names = [os.path.basename(x) for x in filelist_hwp]
hwp_df['title'] = names


content = []
for file in filelist_hwp :
    f = olefile.OleFileIO(file) #파일열기
    encoded_text = f.openstream('PrvText').read() # PrvText 스트림의 내용 꺼내기
    decoded_text = encoded_text.decode('UTF-16') # 유니코드를 UTF-16으로 디코딩
    content.append(decoded_text)

hwp_df['content'] = np.nan
hwp_df['content'] = content
hwp_df['extension'] = "hwp"

# pptx 확장자 dataFrame으로 변환
from pptx import Presentation
filelist_pptx = glob.glob(f'{target_path}/'+'*.pptx')
pptx_df = pd.DataFrame()
names = [os.path.basename(x) for x in filelist_pptx]
for i in filelist_pptx:
  prs = Presentation(f"{i}")

  

  data = []
  for slide in prs.slides:
    
    for shape in slide.shapes:
        if not shape.has_text_frame:
              continue
        text_frame = shape.text_frame
        for paragraph in text_frame.paragraphs:
              for run in paragraph.runs:

                  data.append(run.text)


  pptx_df['title'] = np.nan
  pptx_df['content'] = np.nan
  pptx_df['extension'] = np.nan
  pptx_df["content"] = [listToString(data)]
  pptx_df["title"] = names          
  pptx_df['extension'] = "pptx"

# docx 확장자 dataFrame으로 변환
import docx
filelist_docx = glob.glob(f'{target_path}/'+'*.docx')
docx_df = pd.DataFrame()
names = [os.path.basename(x) for x in filelist_docx]
docx_df['title'] = names
def extract_text_from_docx(file):
    doc = docx.Document(file)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)
content = []
for file in filelist_docx:
    if file.endswith('.docx'):
        extracted_text = extract_text_from_docx(file)
        content.append(extracted_text)
docx_df['content'] = content
docx_df['extension'] = 'docx'


# pdf 사용시 dataFrame으로 변환
from tika import parser
filelist_pdf = glob.glob(f'{target_path}/'+'*.pdf')
names = [os.path.basename(x) for x in filelist_pdf]
pdf_df = pd.DataFrame()

pdf_df['title'] = np.nan
pdf_df['title'] = listToString(names)

pdf_df['content'] = np.nan
content_list = []
for i in filelist_pdf:
  pdf_path = f"{i}"
  parsed = parser.from_file(pdf_path)
  txt = open('output.txt', 'w', encoding = 'utf-8')
  # output.txt에 pdf파일 내용을 write
  content_list.append(parsed['content'])

  txt.close()
pdf_df = pd.DataFrame([names,content_list])
pdf_df = pdf_df.T
pdf_df = pdf_df.rename(columns={0:'title'})
pdf_df = pdf_df.rename(columns={1:'content'})
pdf_df['extension'] = 'pdf'

# xlsx 사용시 dataFrame으로 변환
filelist_xlsx = glob.glob(f'{target_path}/'+'*.xlsx')
names = [os.path.basename(x) for x in filelist_xlsx]
csv_xlsx_df = pd.DataFrame()


for i in filelist_xlsx:
    csv_xlsx_df = pd.read_excel(f"{i}")

full_text = []
for column in csv_xlsx_df:
    for item in csv_xlsx_df[column]:

        full_text.append(str(item))
csv_xlsx_df = csv_xlsx_df.head(1)

csv_xlsx_df['title'] = np.nan
csv_xlsx_df['title'] = listToString(names)
csv_xlsx_df['content'] = listToString(full_text)
csv_xlsx_df['extension'] = 'csv'
csv_xlsx_df = csv_xlsx_df[['title','content','extension']]

In [9]:
df_target_merged = pd.concat([hwp_df, pptx_df, docx_df,pdf_df,csv_xlsx_df])
df_target_merged = df_target_merged.reset_index()
df_target_merged = df_target_merged.drop(['index'],axis=1)

In [10]:
for i in range(len(df_target_merged['content'])):
  df_target_merged['content'].loc[i] = remove_special_characters(df_target_merged['content'].loc[i])

In [11]:
df_target_merged

,title,content,extension
0,제목 없는 문서 (2).docx,Apple Inc는 소비자 전자 제품 컴퓨터 소프트웨어 및 온라인 서비스를 설계 개...,docx
1,제목 없는 문서 (3).docx,카드는 정보나 데이터를 저장하는 데 사용되는 얇고 평평한 재료입니다 카드는 종이 플...,docx


In [12]:
#비교 대상 문서 directory 경로
# hwp 확장자 dataFrame으로 변환
filelist_hwp = glob.glob(f'{compare_path}/'+'*.hwp')


hwp_df = pd.DataFrame()
hwp_df['title'] = np.nan
names = [os.path.basename(x) for x in filelist_hwp]
hwp_df['title'] = names


content = []
for file in filelist_hwp :
    f = olefile.OleFileIO(file) #파일열기
    encoded_text = f.openstream('PrvText').read() # PrvText 스트림의 내용 꺼내기
    decoded_text = encoded_text.decode('UTF-16') # 유니코드를 UTF-16으로 디코딩
    content.append(decoded_text)

hwp_df['content'] = np.nan
hwp_df['content'] = content
hwp_df['extension'] = "hwp"

# pptx 확장자 dataFrame으로 변환
from pptx import Presentation
filelist_pptx = glob.glob(f'{compare_path}/'+'*.pptx')
pptx_df = pd.DataFrame()
names = [os.path.basename(x) for x in filelist_pptx]
for i in filelist_pptx:
    prs = Presentation(f"{i}")



    data = []
    for slide in prs.slides:

        for shape in slide.shapes:
            if not shape.has_text_frame:
                continue
            text_frame = shape.text_frame
            for paragraph in text_frame.paragraphs:
                for run in paragraph.runs:

                    data.append(run.text)


    pptx_df['title'] = np.nan
    pptx_df['content'] = np.nan
    pptx_df['extension'] = np.nan
    pptx_df["content"] = [listToString(data)]
    pptx_df["title"] = names
    pptx_df['extension'] = "pptx"

# docx 확장자 dataFrame으로 변환
import docx
filelist_docx = glob.glob(f'{compare_path}/'+'*.docx')
# target = 'target.docx'
# base = 0
docx_df = pd.DataFrame()
names = [os.path.basename(x) for x in filelist_docx]
docx_df['title'] = names
# index = df[df['title'] == target].index[0]
def extract_text_from_docx(file):
    doc = docx.Document(file)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)
content = []
for file in filelist_docx:
    if file.endswith('.docx'):
        extracted_text = extract_text_from_docx(file)
        content.append(extracted_text)
docx_df['content'] = content
docx_df['extension'] = 'docx'

# # xls 사용시 dataFrame으로 변환
# filelist_xlsx = glob.glob(f'{compare_path}/'+'*.xls')
# names = [os.path.basename(x) for x in filelist_xlsx]
# csv_df = pd.DataFrame()

# full_text = []

# for i in filelist_xlsx:
#     csv_df = pd.read_csv(f"{i}")

# for column in csv_df:
#     for item in csv_df[column]:

#         full_text.append(str(item))



# csv_df['title'] = np.nan
# csv_df['title'] = listToString(names)
# csv_df['content'] = listToString(full_text)
# csv_df['extension'] = 'csv'
# csv_df = pd.DataFrame((csv_df[['title','content','extension']].iloc[len(filelist_xlsx) -1])).T

# pdf 사용시 dataFrame으로 변환
from tika import parser
filelist_pdf = glob.glob(f'{compare_path}/'+'*.pdf')
names = [os.path.basename(x) for x in filelist_pdf]
pdf_df = pd.DataFrame()

pdf_df['title'] = np.nan
pdf_df['title'] = listToString(names)

pdf_df['content'] = np.nan
content_list = []
for i in filelist_pdf:
    pdf_path = f"{i}"
    parsed = parser.from_file(pdf_path)
    txt = open('output.txt', 'w', encoding = 'utf-8')
    # output.txt에 pdf파일 내용을 write
    content_list.append(parsed['content'])

    txt.close()
pdf_df = pd.DataFrame([names,content_list])
pdf_df = pdf_df.T
pdf_df = pdf_df.rename(columns={0:'title'})
pdf_df = pdf_df.rename(columns={1:'content'})
pdf_df['extension'] = 'pdf'

# # xlsx 사용시 dataFrame으로 변환
# filelist_xlsx = glob.glob(f'{compare_path}/'+'*.xlsx')
# names = [os.path.basename(x) for x in filelist_xlsx]
# csv_xlsx_df = pd.DataFrame()


# for i in filelist_xlsx:
#     csv_xlsx_df = pd.read_excel(f"{i}")

# full_text = []
# for column in csv_df:
#     for item in csv_df[column]:

#         full_text.append(str(item))



# csv_xlsx_df['title'] = np.nan
# csv_xlsx_df['title'] = listToString(names)
# csv_xlsx_df['content'] = listToString(full_text)
# csv_xlsx_df['extension'] = 'csv'
# csv_xlsx_df = pd.DataFrame((csv_df[['title','content','extension']].iloc[len(filelist_xlsx) -1])).T

In [13]:
df_compare_merged = pd.concat([hwp_df, docx_df, pptx_df,pdf_df])
df_compare_merged = df_compare_merged.reset_index()
df_compare_merged = df_compare_merged.drop(['index'],axis=1)
for i in range(len(df_compare_merged['content'])):
  df_compare_merged['content'].loc[i] = remove_special_characters(df_compare_merged['content'].loc[i])

In [14]:
df = df_compare_merged

In [15]:

# '*'는 임의 길이의 모든 문자열을 의미
# flist = glob.glob('hwpdata/*.hwp')
# 검사 문서 지정
target = '제목 없는 문서 (2).docx'
# score 척도를 정할 변수를 base로 설정 (ex>표절판별수치 등으로 이용)
base = 0
# empty dataframe 생성
# df = pd.DataFrame()
# 파일 리스트에서 파일이름만 추출하여 리스트로 생성
# names = [os.path.basename(x) for x in flist]


In [16]:
df

,title,content,extension
0,제목 없는 문서 (2).docx,Apple Inc는 소비자 전자 제품 컴퓨터 소프트웨어 및 온라인 서비스를 설계 개...,docx
1,제목 없는 문서 (3).docx,카드는 정보나 데이터를 저장하는 데 사용되는 얇고 평평한 재료입니다 카드는 종이 플...,docx


In [17]:
# 파일명을 title로 갖는 column생성
# df['title'] = names
# 검사 문서의 df상 인덱스값을 index 변수에 저장
index = df[df['title'] == target].index[0]
# 빈 content 열 생성
# content = []
# 코사인 유사도 항목 생성
df['cos_sim'] = np.nan
# 유클리드 거리 항목 생성
df['euc_dis'] = np.nan
# 자카드 유사도 항목 생성
df['jac_sim'] = np.nan

In [18]:
df

,title,content,extension,cos_sim,euc_dis,jac_sim
0,제목 없는 문서 (2).docx,Apple Inc는 소비자 전자 제품 컴퓨터 소프트웨어 및 온라인 서비스를 설계 개...,docx,NaN,NaN,NaN
1,제목 없는 문서 (3).docx,카드는 정보나 데이터를 저장하는 데 사용되는 얇고 평평한 재료입니다 카드는 종이 플...,docx,NaN,NaN,NaN


In [19]:

# for file in flist :
#     # .hwp 파일을 읽기위해 olefile 임포트
#     f = olefile.OleFileIO(file)
#     # 읽어온 .hwp파일을 읽기 전용 파일 개체(BytesIO)로 encoded_text에 저장
#     encoded_text = f.openstream('PrvText').read()
#     # 읽기 전용 파일 개체를 'utf-16'방식으로 decoding
#     decoded_text = encoded_text.decode('UTF-16')
#     # decoding한 값을 content에 list 항목으로 저장
#     content.append(decoded_text)

In [20]:
# df['content'] = content

In [21]:
# df

In [22]:
# df['content'][0]

## 3. 벡터화 및 유사도 측정

In [23]:
# 자연어를 벡터화하기 위한 Tf-idf 적용
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['content'])


In [24]:
# 문서의 각단어별로 문서에 대한 정보를 얼마나 갖고 있느냐를 뜻하는 값
# (a,b)는 array로 만들었을 때 각 뒤에 나오는 tf-idf 값의 위치(단어사전에서의 key값)
print(tfidf_matrix)
# (a,b) a는 target을 포함한 전체문서의 수, b는 전체문서에서 정해진 단어 사전의 key값의 총합
print(tfidf_matrix.shape)
print(tfidf.get_feature_names_out()) # 단어사전


  (0, 22)	0.08768895655734549
  (0, 93)	0.08768895655734549
  (0, 128)	0.08768895655734549
  (0, 21)	0.062391414807429685
  (0, 52)	0.062391414807429685
  (0, 76)	0.062391414807429685
  (0, 80)	0.062391414807429685
  (0, 70)	0.062391414807429685
  (0, 120)	0.062391414807429685
  (0, 48)	0.062391414807429685
  (0, 5)	0.08768895655734549
  (0, 0)	0.17537791311469098
  (0, 18)	0.26306686967203646
  (0, 66)	0.08768895655734549
  (0, 105)	0.08768895655734549
  (0, 63)	0.08768895655734549
  (0, 109)	0.08768895655734549
  (0, 7)	0.08768895655734549
  (0, 14)	0.08768895655734549
  (0, 57)	0.08768895655734549
  (0, 17)	0.08768895655734549
  (0, 13)	0.17537791311469098
  (0, 110)	0.08768895655734549
  (0, 85)	0.08768895655734549
  (0, 9)	0.17537791311469098
  :	:
  (1, 84)	0.07579892499568072
  (1, 41)	0.07579892499568072
  (1, 92)	0.07579892499568072
  (1, 47)	0.07579892499568072
  (1, 95)	0.07579892499568072
  (1, 38)	0.07579892499568072
  (1, 45)	0.15159784999136144
  (1, 125)	0.0757989249956

--------

#### 3-1. 코사인 유사도

In [25]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [26]:
print(cosine_sim)

[[1.         0.02691893]
 [0.02691893 1.        ]]


In [27]:
sim_scores = list(enumerate(cosine_sim[index]))


In [28]:
print(sim_scores)

[(0, 1.000000000000002), (1, 0.026918930361656802)]


In [29]:
for num_content in range(len(df)):
    print(df.iloc[num_content]['title'], sim_scores[num_content][1])
    df['cos_sim'].loc[num_content] = sim_scores[num_content][1]

제목 없는 문서 (2).docx 1.000000000000002
제목 없는 문서 (3).docx 0.026918930361656802


C:\Users\user\AppData\Local\Temp\ipykernel_5828\2181676359.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cos_sim'].loc[num_content] = sim_scores[num_content][1]


In [30]:
# sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
# for key, score in sim_scores :
#     if score > base :
#         print(df.iloc[key]['title'], score)

In [31]:
df

,title,content,extension,cos_sim,euc_dis,jac_sim
0,제목 없는 문서 (2).docx,Apple Inc는 소비자 전자 제품 컴퓨터 소프트웨어 및 온라인 서비스를 설계 개...,docx,1.000000,NaN,NaN
1,제목 없는 문서 (3).docx,카드는 정보나 데이터를 저장하는 데 사용되는 얇고 평평한 재료입니다 카드는 종이 플...,docx,0.026919,NaN,NaN


------------

#### 3-2. 유클리드 거리

In [32]:
# 거리값이 무한정 커지는 것을 방지하기 위한 L1 정규화
# L1 정규화는 각 벡터안의 요소값을 모두 더한 것이 크기가 1이 되도록 벡터들의 크기를 조절하는 방법
def l1_normalize(v):
    norm = np.sum(v)
    return v / norm

In [33]:
tfidf_norm_l1 = l1_normalize(tfidf_matrix)

In [34]:
print(tfidf_norm_l1)

  (0, 22)	0.006136802626477229
  (0, 93)	0.006136802626477229
  (0, 128)	0.006136802626477229
  (0, 21)	0.0043663856121890585
  (0, 52)	0.0043663856121890585
  (0, 76)	0.0043663856121890585
  (0, 80)	0.0043663856121890585
  (0, 70)	0.0043663856121890585
  (0, 120)	0.0043663856121890585
  (0, 48)	0.0043663856121890585
  (0, 5)	0.006136802626477229
  (0, 0)	0.012273605252954458
  (0, 18)	0.018410407879431684
  (0, 66)	0.006136802626477229
  (0, 105)	0.006136802626477229
  (0, 63)	0.006136802626477229
  (0, 109)	0.006136802626477229
  (0, 7)	0.006136802626477229
  (0, 14)	0.006136802626477229
  (0, 57)	0.006136802626477229
  (0, 17)	0.006136802626477229
  (0, 13)	0.012273605252954458
  (0, 110)	0.006136802626477229
  (0, 85)	0.006136802626477229
  (0, 9)	0.012273605252954458
  :	:
  (1, 84)	0.00530469354705394
  (1, 41)	0.00530469354705394
  (1, 92)	0.00530469354705394
  (1, 47)	0.00530469354705394
  (1, 95)	0.00530469354705394
  (1, 38)	0.00530469354705394
  (1, 45)	0.01060938709410788
 

In [35]:
for num_content in range(len(df)):
    dist = euclidean_distances(tfidf_norm_l1[index], tfidf_norm_l1[num_content])
    print(df.iloc[num_content]['title'], np.ravel(dist, order='C')[0])
    df['euc_dis'].loc[num_content] = np.ravel(dist, order='C')[0]

제목 없는 문서 (2).docx 0.0
제목 없는 문서 (3).docx 0.09763078091339847


C:\Users\user\AppData\Local\Temp\ipykernel_5828\743567586.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['euc_dis'].loc[num_content] = np.ravel(dist, order='C')[0]


In [36]:
df

,title,content,extension,cos_sim,euc_dis,jac_sim
0,제목 없는 문서 (2).docx,Apple Inc는 소비자 전자 제품 컴퓨터 소프트웨어 및 온라인 서비스를 설계 개...,docx,1.000000,0.000000,NaN
1,제목 없는 문서 (3).docx,카드는 정보나 데이터를 저장하는 데 사용되는 얇고 평평한 재료입니다 카드는 종이 플...,docx,0.026919,0.097631,NaN


-----------------------

#### 3-3. 자카드 유사도

In [37]:
okt = Okt()

In [38]:
target = okt.morphs(df['content'][index])

In [39]:
for num_content in range(len(df)):
    compare = okt.morphs(df['content'][num_content])
    union = set(target).union(set(compare))
    # print('문서1과 문서2의 합집합 :',union)
    intersection = set(target).intersection(set(compare))
    # print('문서1과 문서2의 교집합 :',intersection)
    # print(f'{num_content}번 text와의 자카드 유사도 :',len(intersection)/len(union))
    jac_score = len(intersection)/len(union)
    print(df.iloc[num_content]['title'], jac_score)
    df['jac_sim'].loc[num_content] = jac_score


제목 없는 문서 (2).docx 1.0
제목 없는 문서 (3).docx 0.16025641025641027


C:\Users\user\AppData\Local\Temp\ipykernel_5828\647782823.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['jac_sim'].loc[num_content] = jac_score


---------------

-----------------

## 4. 최종 OUTPUT

In [40]:
# 최종 데이터프레임 (title, content, 확장자명, cos_sim, euc_dis, jac_sim)
df

,title,content,extension,cos_sim,euc_dis,jac_sim
0,제목 없는 문서 (2).docx,Apple Inc는 소비자 전자 제품 컴퓨터 소프트웨어 및 온라인 서비스를 설계 개...,docx,1.000000,0.000000,1.000000
1,제목 없는 문서 (3).docx,카드는 정보나 데이터를 저장하는 데 사용되는 얇고 평평한 재료입니다 카드는 종이 플...,docx,0.026919,0.097631,0.160256
